In [2]:
import glob
import os
import librosa
import numpy as np

path_old = 'D:\\Research\\About4\\data\\BMD-HS-Dataset-main\\BMD-HS-Dataset-main\\seg_re\\5'
audio = []
wav_files = glob.glob(os.path.join(path_old, '*.wav'))

for file_name in wav_files:
    base_name = os.path.basename(file_name)
    if base_name[0] == 'N':
        label = np.array([0., 1.])   # normal
    else:
        label = np.array([1., 0.])   # abnormal
    
    y, sr = librosa.load(file_name, sr=1000)
    audio.append([base_name, label, y])
    
audio = np.array(audio, dtype=object)
np.save("BMD_audio/audios_and_labels.npy", audio)

In [13]:
import numpy as np

audio = np.load("BMD_audio/audios_and_labels.npy", allow_pickle=True)
np.random.shuffle(audio)

n_total = len(audio)
n_train = int(0.8 * n_total)
n_val = int(0.1 * n_total)
n_test = n_total - n_train - n_val

train_set = audio[:n_train]
val_set = audio[n_train:n_train+n_val]
test_set = audio[n_train+n_val:]

print(f"Total: {n_total}, Train: {len(train_set)}, Val: {len(val_set)}, Test: {len(test_set)}")

np.save("BMD_audio2/train_audios.npy", train_set)
np.save("BMD_audio2/val_audios.npy", val_set)
np.save("BMD_audio2/test_audios.npy", test_set)


Total: 20678, Train: 16542, Val: 2067, Test: 2069


In [6]:
import numpy as np
import librosa

def extract_features_from_waveform(y, sr=1000, n_fft=1024, win_length=50, hop_length=25, n_mels=40):
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, win_length=win_length, hop_length=hop_length, n_mels=n_mels)
    return S

audio_path = "BMD_audio/val_audios.npy"
audio_data = np.load(audio_path, allow_pickle=True)
mel_features = []

for idx, sample in enumerate(audio_data):
    filename, label, y = sample
    mels = extract_features_from_waveform(y, sr=1000)
    mel_features.append([filename, label, y, mels])

mel_features = np.array(mel_features, dtype=object)
np.save("BMD_mel/val_mels_40_100.npy", mel_features)
